## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
!pip install geopy
!pip install pandasql

### Importing required Libraries

In [3]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
address = 'Toronto,CA'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude

longitude = location.longitude
type(location.latitude)
type(location.longitude)
radius='500'
version='20190830'
limit='100'
print('The geograpical coordinate of', address,' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto,CA  are 43.653963, -79.387207.


In [7]:
CLIENT_ID='K4DL2WXO0MWA1GAWSJNCVH24V245LLBKOC1M20YWRR5QMVKF'
CLIENT_SECRET = 'DOI2LBPCPWOEFIPVZ5ATWFSDK0KEMSR1SFLUXF2FUWMWAZKF'

LIMIT = 100
RADIUS= 500
VERSION= 20190830

In [8]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [9]:
results

{'meta': {'code': 200, 'requestId': '5d712d3b2b274a003942bd11'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 78,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [10]:
import math
venues = results['response']['groups'][0]['items']
# #venues
nearby_venues = json_normalize(venues)


In [11]:
df=pd.read_excel('canada.xlsx',sheet_name='toronto')
# Note - the Canada.xlsx spreadsheet was created by me, Jim Keenan, Week 4 of the Capstone Project
df.head(5)


,Unnamed: 0,Postcode,Latitude,Longitude,Borough,Neighborhood
0,0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, West Hill, Morningside"
3,3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [12]:
print('The length of nearby_venues, which become neighborhoods, is',len(nearby_venues))
column_names = ['Postcode', 'Latitude', 'Longitude']#,'Borough','Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

data=0
while data<(len(nearby_venues)):
    try:
        postcode= (nearby_venues['venue.location.formattedAddress'][data][1]).split(' ')[2]
    except:
        
        postcode='MB1' 
        

    neighborhood_lat = nearby_venues['venue.location.lat'][data]
    neighborhood_lon = nearby_venues['venue.location.lng'][data]
    data+=1
    
    neighborhoods = neighborhoods.append({'Postcode': postcode,
#                                           'Borough':borough,
#                                           'Neighborhood':neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon
                                           }, ignore_index=True)
   
neighborhoods


The length of nearby_venues, which become neighborhoods, is 78


,Postcode,Latitude,Longitude
0,MB1,43.653232,-79.385296
1,M5G,43.655268,-79.385165
2,M5T,43.654571,-79.389450
3,M5G,43.655157,-79.386501
4,M5G,43.654895,-79.385052
5,M5G,43.655374,-79.385354
6,M5G,43.654396,-79.386500
7,M5G,43.654918,-79.387424
8,M5T,43.653612,-79.390613
9,M5G,43.655542,-79.384684


### Joining the Canadian zip code database from Week 3 to neighborhoods to get Borough and Neighborhood columns


In [13]:

df
resultDF = pd.merge(df, neighborhoods, on='Postcode', how='inner')
resultDF.head()

neighborhoods=resultDF
neighborhoods
#neighborhoods.drop(['Latitude_x','Longitude_x','Borough_y','Neighborhood_y'],axis=1,inplace=True)

neighborhoods.rename(columns={"Latitude_y": "Latitude", "Longitude_y":'Longitude',"Borough_x":"Borough",\
                               "Neighborhood_x":"Neighborhood"},inplace=True)

 

In [14]:
neighborhoods

,Unnamed: 0,Postcode,Latitude_x,Longitude_x,Borough,Neighborhood,Latitude,Longitude
0,54,M5B,43.657162,-79.378937,Downtown Toronto,"Ryerson, Garden District",43.654248,-79.381232
1,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655268,-79.385165
2,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655157,-79.386501
3,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.654895,-79.385052
4,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655374,-79.385354
5,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.654396,-79.386500
6,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.654918,-79.387424
7,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655542,-79.384684
8,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.654603,-79.386204
9,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.654608,-79.385942


In [15]:
len(neighborhoods)

68

In [16]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 1 boroughs and 68 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [17]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of '+address+' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.653963, -79.387207.


In [18]:
# create map of London using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Greater London. So let's slice the original dataframe and create a new dataframe of the Downtown Toronto data.

In [20]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
manhattan_data.head()

,Unnamed: 0,Postcode,Latitude_x,Longitude_x,Borough,Neighborhood,Latitude,Longitude
0,54,M5B,43.657162,-79.378937,Downtown Toronto,"Ryerson, Garden District",43.654248,-79.381232
1,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655268,-79.385165
2,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655157,-79.386501
3,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.654895,-79.385052
4,57,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street,43.655374,-79.385354


In [21]:
address = 'Downtown Toronto, Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of '+address+' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto, Toronto, Canada are 43.655115, -79.380219.


As we did with all of Toronto, let's visualize Downtown Toronto and the neighborhoods in it.

In [22]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [24]:
CLIENT_ID='K4DL2WXO0MWA1GAWSJNCVH24V245LLBKOC1M20YWRR5QMVKF'
CLIENT_SECRET = 'DOI2LBPCPWOEFIPVZ5ATWFSDK0KEMSR1SFLUXF2FUWMWAZKF'
VERSION = '20190830'
LIMIT = 100
RADIUS=500


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K4DL2WXO0MWA1GAWSJNCVH24V245LLBKOC1M20YWRR5QMVKF
CLIENT_SECRET:DOI2LBPCPWOEFIPVZ5ATWFSDK0KEMSR1SFLUXF2FUWMWAZKF


#### Let's explore the first neighborhood in our dataframe.

In [25]:
manhattan_data.loc[0, 'Neighborhood']

'Ryerson, Garden District'

Get the neighborhood's latitude and longitude values.

In [26]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Ryerson, Garden District are 43.654248, -79.381232.


Get the neighborhood's latitude and longitude values.

First, let's create the GET request URL. Name your URL **url**.

In [27]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=K4DL2WXO0MWA1GAWSJNCVH24V245LLBKOC1M20YWRR5QMVKF&client_secret=DOI2LBPCPWOEFIPVZ5ATWFSDK0KEMSR1SFLUXF2FUWMWAZKF&v=20190830&ll=43.654248,-79.381232&radius=500&limit=100'

In [28]:
results = requests.get(url).json()
#results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Ed Mirvish Theatre,Theater,43.655102,-79.379768
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Indigo,Bookstore,43.653515,-79.380696
4,LUSH,Cosmetics Shop,43.653557,-79.380400


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Greater London

In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [34]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
print("End of Venues!!")


Ryerson, Garden District
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Central Bay Street
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Richmond, Adelaide, King
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
Kensington Market, Chinatown, Grange Park
Kensington Ma

### Checking the size of the resultant dataframe


In [35]:
print(manhattan_venues.shape)
manhattan_venues.head()

(6127, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ryerson, Garden District",43.654248,-79.381232,UNIQLO ユニクロ,43.655910,-79.380641,Clothing Store
1,"Ryerson, Garden District",43.654248,-79.381232,Ed Mirvish Theatre,43.655102,-79.379768,Theater
2,"Ryerson, Garden District",43.654248,-79.381232,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
3,"Ryerson, Garden District",43.654248,-79.381232,Indigo,43.653515,-79.380696,Bookstore
4,"Ryerson, Garden District",43.654248,-79.381232,LUSH,43.653557,-79.380400,Cosmetics Shop


Let's check how many venues were returned for each neighborhood

In [36]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CN Tower, Harbourfront West, King and Spadina, Railway Lands, South Niagara, Island airport, Bathurst Quay",1200,1200,1200,1200,1200,1200
Central Bay Street,2137,2137,2137,2137,2137,2137
"Kensington Market, Chinatown, Grange Park",1690,1690,1690,1690,1690,1690
"Richmond, Adelaide, King",1000,1000,1000,1000,1000,1000
"Ryerson, Garden District",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 126 uniques categories.


## 3. Analyze Each Neighborhood

In [38]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Park,Pharmacy,Pie Shop,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [39]:
manhattan_onehot.shape

(6127, 126)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [40]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Beer Bar,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Costume Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Fast Food Restaurant,Food Court,French Restaurant,Furniture / Home Store,Gaming Cafe,Gastropub,General Travel,Gift Shop,Gluten-free Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Latin American Restaurant,Lingerie Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Park,Pharmacy,Pie Shop,Pizza Place,Plaza,Poke Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"CN Tower, Harbourfront West, King and Spadina,...",0.010000,0.003333,0.010833,0.005833,0.020000,0.017500,0.000000,0.000000,0.021667,0.009167,0.001667,0.014167,0.010000,0.010000,0.018333,0.000000,0.000,0.010000,0.020000,0.048333,0.000000,0.000000,0.010000,0.000000,0.066667,0.000000,0.010000,0.000000,0.019167,0.017500,0.010000,0.000000,0.010000,0.000833,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018333,0.000000,0.005833,0.009167,0.010000,0.010000,0.000000,0.000000,0.005000,0.000000,0.000000,0.009167,0.000000,0.020000,0.008333,0.000000,0.020833,0.000000,0.000000,0.000000,0.010000,0.010833,0.027500,0.010000,0.010000,0.000000,0.000000,0.000000,0.019167,0.000000,0.000000,0.010000,0.010000,0.002500,0.000000,0.010000,0.000833,0.019167,0.002500,0.000000,0.010000,0.010000,0.000000,0.002500,0.010000,0.010000,0.000000,0.000000,0.035833,0.009167,0.000000,0.010000,0.000000,0.010000,0.010000,0.002500,0.010000,0.010000,0.000833,0.003333,0.000000,0.010000,0.000000,0.009167,0.010000,0.000000,0.010000,0.015000,0.037500,0.020000,0.010000,0.000000,0.003333,0.001667,0.015833,0.020000,0.000000,0.010000,0.03000,0.000833,0.000000,0.000000,0.009167
1,Central Bay Street,0.000936,0.014974,0.007487,0.010295,0.003276,0.007955,0.021058,0.007019,0.011699,0.009359,0.000000,0.017314,0.000000,0.000468,0.018250,0.019654,0.000,0.013570,0.006551,0.027141,0.019186,0.000000,0.055686,0.000000,0.100140,0.002808,0.000468,0.008891,0.004679,0.017782,0.000468,0.000000,0.000468,0.000468,0.008423,0.010295,0.00000,0.010763,0.000468,0.001872,0.000000,0.007487,0.000468,0.001404,0.017314,0.003276,0.002340,0.007955,0.000000,0.011231,0.000936,0.000468,0.000468,0.003276,0.000936,0.011699,0.015910,0.000000,0.012635,0.000000,0.000000,0.020122,0.000936,0.019654,0.014038,0.001872,0.001404,0.000936,0.001404,0.007955,0.000936,0.000000,0.000000,0.000468,0.000468,0.031352,0.009827,0.009827,0.007019,0.011231,0.007487,0.001404,0.000468,0.000468,0.010295,0.000000,0.000000,0.006551,0.000000,0.000000,0.008891,0.024801,0.009827,0.002808,0.012167,0.002340,0.015910,0.001404,0.000468,0.007955,0.011699,0.008423,0.009359,0.000936,0.001872,0.000000,0.000468,0.016378,0.000468,0.016846,0.012167,0.014974,0.000936,0.008423,0.0

#### Let's confirm the new size

In [41]:
manhattan_grouped.shape

(5, 126)

#### Let's print each neighborhood along with the top 5 most common venues

In [42]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CN Tower, Harbourfront West, King and Spadina, Railway Lands, South Niagara, Island airport, Bathurst Quay----
                           venue  freq
0                    Coffee Shop  0.07
1                           Café  0.05
2                    Pizza Place  0.04
3                     Steakhouse  0.04
4  Vegetarian / Vegan Restaurant  0.03


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.10
1             Clothing Store  0.06
2                   Tea Room  0.03
3  Middle Eastern Restaurant  0.03
4                       Café  0.03


----Kensington Market, Chinatown, Grange Park----
                 venue  freq
0          Coffee Shop  0.10
1   Chinese Restaurant  0.07
2                 Café  0.06
3       Sandwich Place  0.04
4  Japanese Restaurant  0.04


----Richmond, Adelaide, King----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3                Hotel  0.03
4  Ame

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
import numpy as np
import pandas as pd

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CN Tower, Harbourfront West, King and Spadina,...",Coffee Shop,Café,Steakhouse,Pizza Place,Vegetarian / Vegan Restaurant,Japanese Restaurant,Bar,Hotel,Burrito Place,Arts & Crafts Store
1,Central Bay Street,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Tea Room,Café,Plaza,Bakery,Ice Cream Shop,Bubble Tea Shop,Italian Restaurant
2,"Kensington Market, Chinatown, Grange Park",Coffee Shop,Chinese Restaurant,Café,Japanese Restaurant,Sandwich Place,Ice Cream Shop,Sushi Restaurant,Bar,Art Gallery,Tea Room
3,"Richmond, Adelaide, King",Coffee Shop,Café,Steakhouse,Hotel,Bar,American Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Breakfast Spot,Gym
4,"Ryerson, Garden District",Clothing Store,Coffee Shop,American Restaurant,Plaza,Restaurant,Tea Room,Bakery,Café,Ice Cream Shop,Bubble Tea Shop


## 4. Cluster Neighborhoods

### Run *k*-means to cluster the neighborhood into 5 clusters.

In [46]:
# set number of clusters
kclusters = 5 #Was 3

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] #Was [0:3]

array([3, 4, 2, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

Finally, let's visualize the resulting clusters

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters